In [9]:
import numpy as np
import pandas as pd

import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from pathlib import Path

In [10]:
def msds2ravenpro(input_csv, output_txt):
    df = pd.read_csv(input_csv)

    sels = np.arange(df.shape[0])+1
    view = []
    for sel in sels:
        view.append('Waveform 1')
        view.append('Spectrogram 1')

    rp_columns = ['Selection', 'Channel', 'Begin Time (s)', 'End Time (s)', 'Low Freq (Hz)', 'High Freq (Hz)', 'Detection Confidence', 'Event', 'Input File']
    df.insert(loc = 0, column = rp_columns[1], value=np.ones(df.shape[0]).astype('int16')) # Channel Column
    df.insert(loc = 0, column = rp_columns[0], value=np.arange(1, df.shape[0]+1)) # Selection Column

    #define new DataFrame as original DataFrame with each row repeated 2 times
    df = pd.DataFrame(np.repeat(df.values, 2, axis=0))
    #assign column names of original DataFrame to new DataFrame
    df.columns = rp_columns

    df.insert(loc=1, column = 'View', value=view) # View column

    df.to_csv(output_txt, header=True, index=None, sep='\t', mode='a')

def subsample(percent_on, audio):

    time_on = percent_on*audio.shape[0]
    audio[np.ceil(time_on).astype('int32'):] = 0

    return audio.astype('int16')

In [12]:
filename = '20210910_030000.WAV'
csv_file = f'../output_dir/BatDetectorMSDS-{filename}.csv'
output_txt = f'../output_dir/{filename}.txt'

msds2ravenpro(csv_file, output_txt)

In [8]:
p_ON = 0.5
filepath = f"{Path.home()}/Downloads"

fs, audio = wav.read(Path(f"{filepath}/{filename}"))
wav.write(f'./{int(100*round(p_ON, 2))}_{filename}', fs, subsample(p_ON, audio))

In [14]:
filename = f'{int(100*round(p_ON, 2))}_{filename}'
csv_file = f'BatDetectorMSDS-{filename}.csv'
output_txt = f'{filename}.txt'

msds2ravenpro(csv_file, output_txt)